In [1]:
import redis

In [3]:
REDIS = redis.Redis(host='data_store')

In [4]:
REDIS.get('my_incrementer')

b'42'

In [5]:
#To pass a dictionary:
import numpy as np
import json

model_params = {
    'C': list(np.logspace(-3,3,7)),
    'penalty': 'l1',
    'solver' : 'newton-cg'
}

REDIS.set('model_params', json.dumps(model_params))

True

In [6]:
json.loads(REDIS.get('model_params').decode())

{'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0],
 'penalty': 'l1',
 'solver': 'newton-cg'}

In [ ]:
#Manager Log
# Need to send Number of Companies, Rate